# Starting and testing OVRO-LWA

### Setup

1. Use dashboard to confirm that services are up for etcd, bifrost pipelines, and data capture.
2. Identify server names for snap2, gpu, and data capture.
3. Set list of server names in notebook.
4. Execute cells


### F-eng
* Repo https://github.com/realtimeradio/caltech-lwa
* python install "control_sw"

### X-eng
* Repo https://github.com/realtimeradio/caltech-bifrost-dsp
* python install "pipeline-control"

### mnc
* Repo https://github.com/ovro-lwa/mnc_python
* python install "mnc_python"

In [1]:
snap2names = [f'snap{i:02}' for i in range(1, 12)]  # f-eng
feng_config = '/home/ubuntu/proj/lwa-shell/caltech-lwa/control_sw/config/lwa_corr_config.yaml'  # f-eng and x-eng
xhosts = ['lxdlwagpu02']  # x-eng
npipeline = 4  # x-eng
drips = ['10.41.0.25', '10.41.0.41']  # data recorder (two ips for lxdlwagpu09)
dest_ip = list(sorted(drips*2))  # correlator -> data recorder (destination IPs)
dest_port = [10001+i//npipeline for i in range(npipeline*len(xhosts))]  # correlator -> data recorder (destination ports)
drroot = [f'drvs{ip.split(".")[-1]}' for ip in drips]  # data recorder root names
drids = [f'{drr}{dp-10000:02}' for dp in set(dest_port) for drr in drroot]  # data recorder ids

In [2]:
print(snap2names, xhosts, drips, dest_ip, dest_port, drids)

['snap01'] ['lxdlwagpu02'] ['10.41.0.25', '10.41.0.41'] ['10.41.0.25', '10.41.0.25', '10.41.0.41', '10.41.0.41'] [10001, 10001, 10001, 10001] ['drvs2501', 'drvs4101']


In [3]:
from lwa_f import snap2_fengine
from lwa352_pipeline_control import Lwa352CorrelatorControl
from mnc import mcs, common, ezdr

In [4]:
for snap2name in snap2names:
    lwa_f = snap2_fengine.Snap2Fengine(snap2name)
#    lwa_f.program()
    lwa_f.cold_start_from_config(feng_config)

2021-09-29 23:08:47,425 - lwa_f.blocks.block:snap01 - WARNING -  - hostname (10.40.0.157) couldn't be turned into integer serial
2021-09-29 23:08:47,704 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2021-09-29 23:08:48,004 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1
2021-09-29 23:08:48,020 - lwa_f.snap2_fengine:snap01 - INFO - Trying to configure output with config file /home/ubuntu/proj/caltech-lwa/control_sw/config/lwa_corr_config.yaml
2021-09-29 23:08:48,025 - lwa_f.snap2_fengine:snap01 - INFO - Loading existing firmware to snap01
File in flash is:  snap2_f_200msps_64i_4096c_2021-09-23_1421.fpg
2021-09-29 23:09:13,064 - lwa_f.snap2_fengine:snap01 - INFO - Loaded snap2_f_200msps_64i_4096c_2021-09-23_1421.fpg
2021-09-29 23:09:13,424 - lwa_f.blocks.block:snap01 - WARNING -  - hostname (10.40.0.157) couldn't be turned into integer serial
2021-09-29 23:09:13,759 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board

100% (64 of 64) |########################| Elapsed Time: 0:00:08 Time:  0:00:08


2021-09-29 23:09:39,231 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane delays:
[[336, 328, 336, 312, 320, 312, 312, 312], [320, 312, 304, 304, 304, 304, 304, 304], [296, 296, 296, 296, 296, 288, 296, 304], [312, 320, 304, 304, 304, 304, 304, 304], [296, 296, 296, 304, 304, 296, 296, 296], [296, 304, 304, 296, 288, 296, 288, 288], [328, 304, 312, 312, 312, 320, 312, 312], [368, 360, 360, 360, 360, 376, 376, 360]]
2021-09-29 23:09:39,231 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane slacks:
[[40, 40, 40, 40, 40, 40, 40, 40], [32, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 32, 40, 32], [40, 40, 32, 32, 32, 32, 32, 32], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 32, 40, 40], [32, 32, 32, 32, 32, 40, 32, 40]]
2021-09-29 23:09:39,232 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXX
2021-09-29 23:09:39,232 - lwa_f.blocks.block:snap01 - INF

2021-09-29 23:09:39,255 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 2:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:39,255 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 3:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|-----XXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:39,256 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 4:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:39,256 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 5:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:39,257 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 6:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:39,257 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 7:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:39,258 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 7, Lane 0:    XXXXXXXX

100% (64 of 64) |########################| Elapsed Time: 0:00:08 Time:  0:00:08


2021-09-29 23:09:50,680 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane delays:
[[304, 312, 304, 312, 304, 304, 296, 312], [296, 296, 288, 296, 288, 304, 296, 288], [296, 288, 296, 304, 296, 304, 296, 288], [328, 352, 336, 336, 336, 344, 336, 336], [280, 288, 288, 296, 288, 288, 288, 288], [328, 328, 328, 328, 328, 328, 336, 328], [288, 280, 288, 296, 288, 280, 280, 288], [280, 280, 280, 272, 280, 280, 280, 280]]
2021-09-29 23:09:50,681 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane slacks:
[[40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 32, 32, 40, 32], [40, 40, 40, 32, 40, 32, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40], [32, 32, 32, 32, 32, 32, 32, 32], [32, 32, 40, 40, 40, 32, 32, 40], [40, 40, 32, 40, 40, 40, 40, 40]]
2021-09-29 23:09:50,682 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:50,682 - lwa_f.blocks.block:snap01 - INF

2021-09-29 23:09:50,712 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 2:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:50,713 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 3:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:50,713 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 4:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:50,714 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 5:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:50,714 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 6:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:50,715 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 7:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXX
2021-09-29 23:09:50,715 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 7, Lane 0:    XXXXXXXX

In [5]:
lwa_corr = Lwa352CorrelatorControl(xhosts, npipeline_per_host=npipeline)
lwa_corr.stop_pipelines()   # maybe not needed?
lwa_corr.start_pipelines() 
lwa_corr.configure_corr(dest_ip=dest_ip, dest_port=dest_port)

2021-09-29 23:11:26 [WARNING ] Timeout waiting for pipelines to come up after 60.0 seconds
2021-09-29 23:11:26 [INFO    ] Setting max data output rate per pipeline to 5000.0 Mbit/s
2021-09-29 23:11:26 [INFO    ] Setting pipeline lxdlwagpu02:0 data destination to 10.41.0.25:10001
2021-09-29 23:11:46 [INFO    ] Setting pipeline lxdlwagpu02:1 data destination to 10.41.0.25:10001
2021-09-29 23:11:56 [INFO    ] Setting pipeline lxdlwagpu02:2 data destination to 10.41.0.41:10001
2021-09-29 23:11:56 [INFO    ] Setting pipeline lxdlwagpu02:3 data destination to 10.41.0.41:10001
2021-09-29 23:11:56 [INFO    ] Arming correlator core
2021-09-29 23:11:57 [INFO    ] All pipelines in 'waiting' state as expected
2021-09-29 23:11:57 [INFO    ] Waiting 10 seconds for trigger
2021-09-29 23:12:07 [INFO    ] All pipelines in 'running' state as expected
2021-09-29 23:12:07 [INFO    ] Arming correlator accumulator core
2021-09-29 23:12:08 [INFO    ] All pipelines in 'waiting' state as expected
2021-09-29 23

In [6]:
rs = ezdr.Lwa352RecorderControl('slow')
rs.start()

Found 0 slow recorders: 


[]

### Read data and visualize/test

## Stop

In [9]:
rs.stop()
lwa_corr.stop_pipelines()